# 시계열 모형 예측력 평가
- 작성자: 고려대학교 경제학과 한치록 교수, 데이터사이언스팀 이창훈 과장

시계열 모형의 예측력 평가를 위해 `bok_da` 라이브러리에 여러 기능을 제공한다.

* `col_evaluation_metrics` (RMSE, MAE 등 예측력 정보기준 계산), `dm_test` (Diebold-Mariano test), `cw_test` (Clark-West test) 구현하였다.

* 전망모형 개발 시 검증을 할때 다양한 예측력 평가 기준과 모형의 예측력 평가를 위한 `dm_test`, `cw_test`가 유용하게 활용될 수 있다.

참고로, Workspace를 이용해서도 예측력 평가 분석을 수행할 수 있다.(자세한 내용은 Wokrspace 매뉴얼 참고)

[3_06]: 06%20BOK%20Library%20I%20(LS).ipynb
[3_07]: 07%20BOK%20Library%20II%20(IV).ipynb
[3_08]: 08%20BOK%20Library%20III%20(Time%20Series).ipynb

# 예측성능

## 예측성능 관련 요약통계량

In [1]:
# 라이브러리 호출
import pandas as pd

예측성능 테스트를 위해서 임의로 10개의 실제값 `y`와 4개의 모형에 대한 `f1`, `f2`, `f3`, `f4` 예측치를 생성하였다.

In [2]:
df = pd.read_excel('../data/dmtest.xlsx')
df = df.set_index('id')
df

,y,f1,f2,f3,f4
id,,,,,
0,6.624345,6.916767,5.862317,6.559862,6.547534
1,4.388244,4.614997,3.818265,4.353758,4.212672
2,4.471828,4.480271,4.238391,4.251704,4.700773
3,3.927031,4.107350,3.677530,4.107203,3.790286
4,5.865408,5.840830,5.328254,5.811830,5.971479
5,2.698461,2.560129,2.269111,2.561027,2.529420
6,6.744812,6.610563,6.392279,6.521350,6.791695
7,4.238793,4.570754,4.037202,4.200426,4.061267
8,5.319039,5.169607,5.307530,5.329201,5.191640


이와 같이 실제값 `y`와 비교모형의 예측치가 주어져 있을 때, `bok_da` 라이브러리 - 모형검증 패키지(`validation`) - 예측성능 평가 서브패키지(`pred_perf`)의 `col_evaluation_metrixs` 함수를 이용해서 예측모형에 대한 `MSE`, `RMSE`, `MAE`, `MAPE`, `MPE`, `Rsq` 예측력 평가 기준을 계산할 수 있다.

In [3]:
from bok_da.valid.pred_perf import col_evaluation_metrics

`print_res=True`로 설정할 경우, 다음 셀과 같이 예측력 정보 기준 계산 결과를 출력한다.

In [4]:
res = col_evaluation_metrics(df.y, df.f1, df.f2, df.f3, df.f4, print_res=True)

 Predictor |      MSE      RMSE       MAE      MAPE%      MPE%      Rsq 
-----------+-------------------------------------------------------------
  Model 1  |   .034126   .184733   .152467   3.33425   1.26523   .975938
  Model 2  |   .166514   .408061   .341713   7.28083  -6.98591   .882591
  Model 3  |   .016011   .126535    .09863   2.21914  -1.16216   .988711
  Model 4  |   .021285   .145893   .136843   3.17119  -1.12686   .984992


출력된 결과를 보면 `RMSE`와 `MAE` 기준 `Model 3`, `Model 4`, `Model 1`, `Model 2` 순서로 예측력이 좋은 것으로 나타난다. 이 결과를 다음의 `Diebold-Mariano` 테스트와 `Clark-West` 테스트를 통해 모형 간 예측력 차이가 통계적으로 유의미한 것인지 검증할 수 있다.

## Diebold-Mariano Test

Diebold-Mariano (DM) 테스트는 두 모형의 예측성능을 비교하기 위한 통계적 검증 기법이다. 모형의 예측력을 평가할 때, 단순히 `MAE`, `RMSE` 등의 예측력 평가 기준만으로 비교하는 것이 충분하지 않을 수 있다. DM 테스트는 두 모형의 예측오차를 비교하여 그 차이가 통계적으로 유의미한지 평가한다.

DM 테스트의 검정 방식은 두 모형의 성능 차이가 양쪽 방향(첫 번째 모형이 더 나은지, 두 번째 모형이 더 나은지)를 모두 고려하는 `two-sided`와 특정 방향(첫 번째 모형이 더 뛰어남)에 대해서만 검정을 수행하는 `one-sided(less, greater)`가 있다.

- **two-sided 검정**
  - `귀무가설(H0)`: 두 모형의 예측성능이 동일하다. 즉, 두 모형의 예측오차 차이가 통계적으로 유의미하지 않다.
  - `대립가설(H1)`: 두 모형의 예측성능이 다르다. 즉, 두 모형의 예측오차 차이가 통계적으로 유의미하다.
- **one-sided(greater) 검정**
  - `귀무가설(H0)`: 기준 모형의 예측오차가 비교모형보다 작거나 같다.
  - `대립가설(H1)`: 기준 모형의 예측오차가 비교모형보다 더 크다.(기준모형의 예측력이 더 나쁘다.)
- **one-sided(less) 검정**
  - `귀무가설(H0)`: 기준 모형의 예측오차가 비교모형보다 크거나 같다.
  - `대립가설(H1)`: 기준 모형의 예측오차가 비교모형보다 더 작다.(기준모형의 예측력이 더 뛰어나다.)

`bok_da` 라이브러리 - 모형검증 패키지(`validation`) - 예측성능 평가 서브패키지(`pred_perf`)에 DM 테스트를 위한 `diebold_mariano` 함수를 제공한다.

In [5]:
from bok_da.valid.pred_perf import diebold_mariano

Model 1, Model 2, Model 3, Model 4의 예측치 f1, f2, f3, f4를 이용해 DM 테스트를 해본다. 참고로, `diebold_mariano` 함수도 `print_res=True` 인자를 통해 결과를 출력할 것인지 설정할 수 있다. `alternative` 인자에 `two-sided`, `greater`, `less`를 입력해 검정방식을 설정할 수 있다. 기본값은 `two-sided`로 alternative 인자를 입력하지 않는 경우, two-sided로 테스트한다.

In [6]:
dm = diebold_mariano(df.y, df.f1, df.f2, df.f3, df.f4, h=1, print_res=True)

Diebold-Mariano tests with 
  alternative = two_sided, horizon = 1        , power = 2        
-----------------------------------------------------------
 DM test |        f1          f2          f3          f4   
---------+-------------------------------------------------
      f1 |              -2.39329**   1.17095     .997537   
      f2 |   2.39329**               2.49445**   2.41924** 
      f3 |  -1.17095    -2.49445**              -.735368   
      f4 |  -.997537    -2.41924**   .735368               
-----------------------------------------------------------
note. ***p<0.01, **p<0.05, *p<0.10


위에 출력된 테이블은 DM 테스트 결과를 나타낸다. 행을 기준으로 각 열은 해당 행 모형 대비 다른 모형과의 예측력을 비교평가한 것이다. DM 통계량이 음수이면, 행 모형의 예측력이 비교모형보다 높다는 것을 의미한다. DM 통계량의 `***`는 1% 유의수준, `**`는 5% 유의수준, `*`는 10% 유의수준에서 통계적으로 유의미하다는 것을 말한다. `two-sided` 결과에서 `f3`는 나머지 모형 대비 뛰어나지만 통계적으로는 `f2` 대비해서만 5% 유의수준에서 유의미한 것으로 나타났다. `horizon`은 예측시계를 말한다. 주어진 예측치가 예측시계가 2인 경우이면 `h=2`로 인자를 설정하면 된다. `power = 2`는 두 모형의 오차 차이에 대한 제곱을 의미한다.

`alternative='greater'`로 설정하는 경우 결과는 다음과 같다.

In [7]:
dm = diebold_mariano(df.y, df.f1, df.f2, df.f3, df.f4, alternative='greater', print_res=True)

Diebold-Mariano tests with 
  alternative = greater, horizon = 1      , power = 2      
-----------------------------------------------------------
 DM test |        f1          f2          f3          f4   
---------+-------------------------------------------------
      f1 |              -2.39329     1.17095     .997537   
      f2 |   2.39329**               2.49445**   2.41924** 
      f3 |  -1.17095    -2.49445                -.735368   
      f4 |  -.997537    -2.41924     .735368               
-----------------------------------------------------------
note. ***p<0.01, **p<0.05, *p<0.10


`alternative='less'`로 설정하는 경우 결과는 다음과 같다.

In [8]:
dm = diebold_mariano(df.y, df.f1, df.f2, df.f3, df.f4, alternative='less', print_res=True)

Diebold-Mariano tests with 
  alternative = less, horizon = 1   , power = 2   
-----------------------------------------------------------
 DM test |        f1          f2          f3          f4   
---------+-------------------------------------------------
      f1 |              -2.39329**   1.17095     .997537   
      f2 |   2.39329                 2.49445     2.41924   
      f3 |  -1.17095    -2.49445**              -.735368   
      f4 |  -.997537    -2.41924**   .735368               
-----------------------------------------------------------
note. ***p<0.01, **p<0.05, *p<0.10


`alternative='less'`, `h=2`로 설정하는 경우 결과는 다음과 같다.

In [9]:
dm = diebold_mariano(df.y, df.f1, df.f2, df.f3, df.f4, alternative='less', h=2, print_res=True)

Diebold-Mariano tests with 
  alternative = less, horizon = 2   , power = 2   
-----------------------------------------------------------
 DM test |        f1          f2          f3          f4   
---------+-------------------------------------------------
      f1 |              -1.65418*    1.12943     .873969   
      f2 |   1.65418                 1.76483     1.71795   
      f3 |  -1.12943    -1.76483*               -1.00079   
      f4 |  -.873969    -1.71795*    1.00079               
-----------------------------------------------------------
note. ***p<0.01, **p<0.05, *p<0.10


## Clark-West Test

In [10]:
from bok_da.valid.pred_perf import clark_west

In [11]:
cw = clark_west(df.y, df.f1, df.f2, df.f3, df.f4, print_res=True)

Clark-West tests
-----------------------------------------------------------
 CW test |        f1          f2          f3          f4   
---------+-------------------------------------------------
      f1 |               1.95896**   2.06201**   2.47846***
      f2 |   2.65164***              2.70339***  2.86028***
      f3 |   2.47051*** -.879912                 1.86685** 
      f4 |    3.4219***  .176834     2.96301***            
-----------------------------------------------------------
note. ***p<0.01, **p<0.05, *p<0.10
